In [1]:
CHECKPOINT_DIR = "hdfs://namenode:8020/spark/checkpoint"

DAY_TYPE_WEEKDAY = 0
DAY_TYPE_WEEKEND = 1
DATA_ACTUAL_TIMEZONE = "America/Los_Angeles"

BOOTSTRAP_SERVER = "kafka:29092"
TOPIC = "buses-location"

POSTGRES_URL = "jdbc:postgresql://timescaledb:5432/lametro"
POSTGRES_TABLE_BUS_VELOCITY = "bus_velocity"
POSTGRES_TABLE_BUS_ARRIVAL = "bus_arrival"
POSTGRES_USERNAME = "postgres"
POSTGRES_PASSWORD = "8zr7E3SV"

REDIS_HOST = "redis"
REDIS_PORT = "6379"
REDIS_PASSWORD = "8zr7E3SV"

STATIC_DATA_DIR = "hdfs://namenode:8020/ola/static_data/"
HISTORICAL_DATA_DIR = "hdfs://namenode:8020/ola/historical_data/"
AGGREGATED_DATA_DIR = "hdfs://namenode:8020/ola/aggregated_data/"
TEMP_DIR = "hdfs://namenode:8020/temp"

LOCAL_STATIC_DATA_DIR = "/home/data/static_data/"
LOCAL_HISTORICAL_DATA_DIR = "/home/data/historical_data/"
LOCAL_AGGREGATED_DATA_DIR = "/home/data/aggregated_data/"
LOCAL_TEMP_DIR = "/home/data/temp/"

In [2]:
import os
import sys
import re

from pyspark.sql import SparkSession, SQLContext
from pyspark import  SparkContext, SparkConf
from pyspark.streaming import StreamingContext

# Spark session & context
conf = SparkConf()
conf.setMaster("spark://0.0.0.0:7077").setAppName("hello-world")
conf.set("spark.cores.max", "2")
conf.set("spark.default.parallelism", "2")
conf.set("spark.driver.extraClassPath", "/usr/local/spark/third-party-jars/*")
conf.set("spark.executor.extraClassPath", "/usr/local/spark/third-party-jars/*")
conf.set("spark.sql.caseSensitive", "true")
conf.set("spark.ui.port", "4040")
conf.set("spark.redis.host", REDIS_HOST)
conf.set("spark.redis.port", REDIS_PORT)
conf.set("spark.redis.auth", REDIS_PASSWORD)
conf.set("spark.sql.legacy.allowUntypedScalaUDF", "true")
conf.set("spark.streaming.backpressure.enabled", "true")
conf.set("spark.streaming.receiver.maxRate", "100")


sc = SparkContext(conf=conf)
spark = SparkSession(sc)
sqlContext = SQLContext(sc)
ssc = StreamingContext(sc, 1)

In [3]:
sc.version

'3.1.1'

In [4]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
.container { 
    width:95% !important; 
}
</style>

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

from datetime import datetime
from pytz import timezone
import math

# Read Kafka

In [6]:
# Subscribe to 1 topic defaults to the earliest and latest offsets
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe", "buses-location") \
  .load()

df = df\
    .withColumn('key', df.key.cast(StringType()))\
    .withColumn('value', df.value.cast(StringType()))

In [7]:
schema = StructType([
    StructField("route_id", StringType(), True),
    StructField("id", StringType(), True),
    StructField("run_id", StringType(), True),
    StructField("predictable", BooleanType(), True),
    StructField("seconds_since_report", LongType(), True),
    StructField("heading", DoubleType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

In [8]:
df_1 = df\
    .select(col("timestamp").cast("long"), from_json("value", schema).alias("bus_location"))\
    .select("timestamp", "bus_location.*")

df_1.printSchema()

root
 |-- timestamp: long (nullable = true)
 |-- route_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- run_id: string (nullable = true)
 |-- predictable: boolean (nullable = true)
 |-- seconds_since_report: long (nullable = true)
 |-- heading: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [9]:
df_2 = df_1\
    .withColumn("route_id", col("route_id").cast("int"))\
    .withColumn("direction", substring("run_id", -1, 1).cast("int"))\
    .withColumn("day_type", lit(0))

df_2.printSchema()

root
 |-- timestamp: long (nullable = true)
 |-- route_id: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- run_id: string (nullable = true)
 |-- predictable: boolean (nullable = true)
 |-- seconds_since_report: long (nullable = true)
 |-- heading: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- direction: integer (nullable = true)
 |-- day_type: integer (nullable = false)



In [10]:
broadcastWrapper = sc._jvm.com.github.lhvubtqn.spark.BroadcastWrapper.getInstance()
broadcastWrapper.updateBroadcastVar(sc._jsc, spark._jsparkSession, AGGREGATED_DATA_DIR + "/bus_velocities.json")

In [11]:
from pyspark.sql.column import Column, _to_java_column, _to_seq

_get_velocity = broadcastWrapper.getVelocityUDF() 

def get_velocity(route_id, direction, day_type):
    return Column(_get_velocity.apply(_to_seq(sc, [route_id, direction, day_type], _to_java_column)))

In [12]:
# df_3 = df_2\
#     .withColumn("velocity", get_velocity("route_id", "direction", "day_type"))

# df_3.printSchema()

In [13]:
def processBatch(batch_df, batch_id):
    broadcastWrapper.updateBroadcastVar(sc._jsc, spark._jsparkSession, AGGREGATED_DATA_DIR + "/bus_velocities.json")
    
    batch_df\
        .withColumn("velocity", get_velocity("route_id", "direction", "day_type"))\
        .show(1000, False)
    
writer = df_2\
    .writeStream\
    .foreachBatch(processBatch)\
    .queryName("test")\
    .start()

In [14]:
# write.stop()

+---------+--------+---+------+-----------+--------------------+-------+--------+---------+---------+--------+--------+
|timestamp|route_id|id |run_id|predictable|seconds_since_report|heading|latitude|longitude|direction|day_type|velocity|
+---------+--------+---+------+-----------+--------------------+-------+--------+---------+---------+--------+--------+
+---------+--------+---+------+-----------+--------------------+-------+--------+---------+---------+--------+--------+

+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id   |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443133|460     |5826 |460_201_1 |true       |15                  |345.

+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id   |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443133|207     |8791 |207_286_1 |true       |42                  |182.0  |34.0259294|-118.308962 |1        |0       |{4.643602417340794, 1}  |
|1625443133|210     |6148 |210_295_1 |true       |68                  |163.0  |34.104622 |-118.330177 |1        |0       |{3.947228923362753, -1} |
|1625443133|164     |8324 |164_60_1  |true       |15                  |271.0  |34.1847727|-118.346083 |1        |0       |{1.8648980930325798, -1}|
|1625443133|180     |1712 |180_213_1 |true       |15                  |90.0   |34.145924 |-118.120789 |1        

+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id   |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443163|460     |5826 |460_201_1 |true       |46                  |345.0  |33.907196 |-118.082603 |1        |0       |{4.7736534135847455, 1} |
|1625443163|217     |5832 |217_278_1 |true       |46                  |220.0  |34.101841 |-118.292641 |1        |0       |{3.637564657086309, -1} |
|1625443163|102     |5847 |102_67_0  |true       |20                  |89.0   |34.018137 |-118.284201 |0        |0       |{4.828854107120708, -1} |
|1625443163|14      |5853 |14_500_1  |true       |126                 |339.0  |34.074509 |-118.378296 |1        

+----------+--------+----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id  |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443163|92      |1799|92_268_0  |true       |20                  |359.0  |34.0988503|-118.2592033|0        |0       |{4.258910836646221, -1} |
|1625443163|686     |3977|686_4_1   |true       |20                  |270.0  |34.1458893|-118.1447828|1        |0       |{4.796418184749741, 1}  |
|1625443163|55      |3982|55_287_1  |true       |46                  |220.0  |34.0619682|-118.24658  |1        |0       |{3.605575822322252, 1}  |
|1625443163|603     |3989|603_27_1  |true       |99                  |90.0   |34.146832 |-118.258034 |1        |0     

+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id   |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443178|460     |5826 |460_201_1 |true       |57                  |345.0  |33.907196 |-118.082603 |1        |0       |{4.7736534135847455, 1} |
|1625443178|217     |5832 |217_278_1 |true       |57                  |220.0  |34.101841 |-118.292641 |1        |0       |{3.637564657086309, -1} |
|1625443178|102     |5847 |102_67_0  |true       |31                  |89.0   |34.018137 |-118.284201 |0        |0       |{4.828854107120708, -1} |
|1625443178|14      |5853 |14_500_1  |true       |136                 |339.0  |34.074509 |-118.378296 |1        

...

+----------+--------+----+---------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id  |run_id   |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+----+---------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443283|164     |4090|164_61_0 |true       |2                   |90.0   |34.1865166|-118.4908598|0        |0       |{1.4849783740252864, 1} |
|1625443283|79      |1824|79_497_0 |true       |2                   |64.0   |34.0550515|-118.223856 |0        |0       |{4.435713639275049, 1}  |
|1625443283|111     |8478|111_358_0|true       |28                  |90.0   |33.974506 |-118.3091645|0        |0       |{4.36158295107281, -1}  |
|1625443283|206     |1887|206_172_1|true       |214                 |269.0  |34.095627 |-118.29097  |1        |0       |{7.2

+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id   |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443298|460     |5826 |460_201_1 |true       |181                 |345.0  |33.907196 |-118.082603 |1        |0       |{4.7736534135847455, 1} |
|1625443298|217     |5832 |217_278_1 |true       |181                 |220.0  |34.101841 |-118.292641 |1        |0       |{3.637564657086309, -1} |
|1625443298|102     |5847 |102_67_0  |true       |22                  |30.0   |34.018436 |-118.282158 |0        |0       |{4.828854107120708, -1} |
|1625443298|14      |5853 |14_500_1  |true       |75                  |339.0  |34.074535 |-118.378296 |1        

+----------+--------+----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id  |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443298|207     |8791|207_286_1 |true       |102                 |180.0  |34.0203298|-118.3089438|1        |0       |{4.643602417340794, 1}  |
|1625443298|210     |6148|210_295_1 |true       |234                 |163.0  |34.104622 |-118.330177 |1        |0       |{3.947228923362753, -1} |
|1625443298|164     |8324|164_60_1  |true       |22                  |315.0  |34.1855373|-118.3609077|1        |0       |{1.8648980930325798, -1}|
|1625443298|180     |1712|180_213_1 |true       |22                  |157.0  |34.1458672|-118.115795 |1        |0     

+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id   |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+-----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443313|460     |5826 |460_201_1 |true       |6                   |345.0  |33.914154 |-118.081787 |1        |0       |{4.7736534135847455, 1} |
|1625443313|217     |5832 |217_278_1 |true       |6                   |220.0  |34.101871 |-118.292641 |1        |0       |{3.637564657086309, -1} |
|1625443313|102     |5847 |102_67_0  |true       |6                   |28.0   |34.0216972|-118.2800318|0        |0       |{4.828854107120708, -1} |
|1625443313|14      |5853 |14_500_1  |true       |86                  |339.0  |34.074535 |-118.378296 |1        

+----------+--------+----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|timestamp |route_id|id  |run_id    |predictable|seconds_since_report|heading|latitude  |longitude   |direction|day_type|velocity                |
+----------+--------+----+----------+-----------+--------------------+-------+----------+------------+---------+--------+------------------------+
|1625443313|210     |5703|210_294_0 |true       |6                   |358.0  |33.9152318|-118.3264368|0        |0       |{2.6372118052353843, 1} |
|1625443313|212     |5741|212_213_1 |true       |59                  |195.0  |34.0575543|-118.3460138|1        |0       |{3.2209583460677433, -1}|
|1625443313|108     |5734|108_313_0 |true       |33                  |136.0  |33.9835445|-118.394826 |0        |0       |{5.229467536150746, -1} |
|1625443313|110     |5762|110_238_0 |true       |86                  |329.0  |33.984802 |-118.399796 |0        |0     